In [1]:
import tenseal.sealapi as seal
from polynomial import enc_poly
import util

In [2]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
poly_modulus_degree = 2**14
mod_pow = 35
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [60,mod_pow, mod_pow,mod_pow,mod_pow,mod_pow,mod_pow,mod_pow,60]))
scale = pow(2.0, mod_pow)

# What are the specific primes in the modulus chain? These help calculate exact scales 
# of rescaled ciphertexts later on in this notebook. 
primes = [modulus.value() for modulus in parms.coeff_modulus()]

context = seal.SEALContext(parms,True,seal.SEC_LEVEL_TYPE.TC128)

util.print_parameters(context)

print("|\t Max Bit Count: " + str(seal.CoeffModulus.MaxBitCount(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128)))

/
|Encryption parameters: 
|	scheme: CKKS
|	poly_modulus_degree: 16384
|	coeff_modulus_size: 365 (60 35 35 35 35 35 35 35 60) bits
|	 Max Bit Count: 438


All keys geneated using `seal::KeyGenerator`. We also need an `encoder`, `evaluator`, `encryptor`, and `decryptor`. 

In [3]:

keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

galois_keys = seal.GaloisKeys()
keygen.create_galois_keys()

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

encoder = seal.CKKSEncoder(context)
slot_count = seal.CKKSEncoder.slot_count(encoder)

print('Number of slots: ' + str(slot_count))

Number of slots: 8192


Let $x$ be the encrypted difference between two number in $a,b\in[0,1]$, such that $x\in[-1,1]$.

In [4]:
a=0.2;b=0.8

x = a-b
x_plain = seal.Plaintext()
encoder.encode(x,scale, x_plain)

x_enc = seal.Ciphertext()
encryptor.encrypt(x_plain, x_enc)

The polynomial we want to calculate is
* $g(x) = (35x-35x^3+21x^5-5x^7)/2^4$

which is done using the function `enc_poly` from the `polynomial` module. 

In [5]:
coeffs = [35, -35, 21, -5]
degrees = [1, 3, 5, 7]
power = 4

enc_compare = enc_poly(x_enc, coeffs, degrees, power, evaluator, context, encoder,  parms, relin_keys, encryptor)

	Enc. result:	-1.3125002771397094
	Plain result:	-1.3125000000000002
	Scale:	549769445602.0032
	Ch. ind:	5
	Enc. result:	0.4725000070435594
	Plain result:	0.47250000000000025
	Scale:	549803526483.5916
	Ch. ind:	4
	Enc. result:	-0.10206028426260469
	Plain result:	-0.10206000000000007
	Scale:	549853341343.267
	Ch. ind:	3
	Enc. result:	0.008747992554337947
	Plain result:	0.00874800000000001
	Scale:	549887427425.6552
	Ch. ind:	3


In [6]:
util.print_info(enc_compare, decryptor, context, encoder, (35*x-35*x**3+21*x**5-5*x**7)/2**4)

	Enc. result:	-0.9330967234166608
	Plain result:	-0.933312
	Scale:	549887427425.6552
	Ch. ind:	3
